In [1]:
from pynq.overlays.base import BaseOverlay
from pynq_peripherals import PmodGroveAdapter, ArduinoSEEEDGroveAdapter,ArduinoDIGILENTGroveAdapter
from dfrobot import PynqCar
base = BaseOverlay('base.bit')

In [14]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time

In [17]:
#外设定义
adapter_A = PmodGroveAdapter(base.PMODA, G4='geared_motor')
adapter_B = PmodGroveAdapter(base.PMODB, G4='geared_motor',G1='grove_line_finder',G2='grove_line_finder')
ARDUINO_adapter = ArduinoDIGILENTGroveAdapter(base.ARDUINO, G1='grove_usranger')
right_motor = adapter_A.G4
left_motor = adapter_B.G4
left_line_finder = adapter_B.G1
right_line_finder = adapter_B.G2
usranger = ARDUINO_adapter.G1
#外设初始化
right_motor.backward()
left_motor.backward()
right_motor.set_speed(0)
left_motor.set_speed(0)

In [19]:
# 传感器测试
print('\r' + 'distance: {:.2f} cm'.format(usranger.get_distance())+'left_line_finder:{}'.format(left_line_finder.line_found())+'right_line_finder:{}'.format(right_line_finder.line_found()), end='', flush=True)

distance: 19.33 cm
False
False


In [ ]:
# 小车 Class 定义
class PynqCar():
    def __init__(self, left_motor, right_motor):
        self.left_motor = left_motor
        self.right_motor = right_motor
        self.speed = 0
        self.speed_offset = 0
        self.speed_diff = 0
        self.stop()
        
    def set_speed(self, speed):
        if (abs(speed)+abs(self.speed_offset)+abs(self.speed_diff) < 100):
            self.speed = speed
            left_speed = self.speed+self.speed_offset+self.speed_diff
            right_speed = self.speed-self.speed_offset-self.speed_diff
            print('\r' + 'left:{}'.format(left_speed)+'right:{}'.format(right_speed), end='', flush=True)
            if left_speed < 0:
                self.left_motor.backward()
                self.left_motor.set_speed(0-left_speed)
            else:
                self.left_motor.forward()
                self.left_motor.set_speed(left_speed)
            if right_speed < 0:
                self.right_motor.backward()
                self.right_motor.set_speed(0-right_speed)
            else:
                self.right_motor.forward()
                self.right_motor.set_speed(right_speed)
        else:
            print("Error: Speed must be -100 to 100")
            
    def stop(self):
        self.right_motor.set_speed(0)
        self.left_motor.set_speed(0)
        
    def steering(self,diff,speed):
        self.speed_diff = diff
        self.set_speed(speed)
    
    def trun_left(self,speed,low=0):
        self.left_motor.backward()
        self.right_motor.forward()
        self.left_motor.set_speed(low)
        self.right_motor.set_speed(speed)
        
    def trun_right(self,speed,low=0):
        self.left_motor.forward()
        self.right_motor.backward()
        self.left_motor.set_speed(speed)
        self.right_motor.set_speed(low)    
        
    def left(self,t,speed):
        self.left_motor.backward()
        self.right_motor.forward()
        self.left_motor.set_speed(speed)
        self.right_motor.set_speed(speed)
        time.sleep(t)
        self.set_speed(self.speed)
        
    def right(self,t,speed):
        self.left_motor.forward()
        self.right_motor.backward()
        self.left_motor.set_speed(speed)
        self.right_motor.set_speed(speed)
        time.sleep(t)
        self.set_speed(self.speed)

In [24]:
car = PynqCar(left_motor, right_motor)

In [30]:
#固定误差调试
def stright_test(speed):
    car.set_speed(speed)
    time.sleep(3)
    car.stop()
    
btn = widgets.Button(description='stright_test')
display(btn)
def btn_eventhandler(obj):
    stright_test(50)
    
btn.on_click(btn_eventhandler)


left:0right:0

In [ ]:
#左右90度转弯调试
btn1 = widgets.Button(description='Left_Test')
btn2 = widgets.Button(description='Right_Test')
display(btn1)
display(btn2)
def btn1_eventhandler(obj):
    car.left(0.3,91)

def btn2_eventhandler(obj):
    car.right(0.3,87)
    
btn1.on_click(btn1_eventhandler)
btn2.on_click(btn2_eventhandler)

In [ ]:
#中间巡线方法
def Navigation_B(B_diff,B_speed,threshold):
    car.speed_offset = -1
    car.speed_diff = 0
    car.set_speed(B_speed)
    flag = 0
    while True:
            left = left_line_finder.line_found()
            right = right_line_finder.line_found()
            if flag > threshold:
                print('FOUND BlackLine---> convert to mode A')
                car.stop()
                break
            elif not left and  not right :
                car.set_speed(B_speed)
            elif left and not right:
                car.trun_left(B_diff)
            elif not left and right:
                car.trun_right(B_diff)
            elif left and  right:
                flag = flag + 1 
            else:
                car.stop()

In [ ]:
#两边巡线方法
def Navigation_A(A_diff,A_speed,distance_T,threshold,turnlist=['right','left']):
    car.speed_offset = -1
    car.speed_diff = 0
    car.set_speed(A_speed)
    flag = 0
    turn_number = 0
    Final = False
    while True:
        distance = usranger.get_distance()
        left = left_line_finder.line_found()
        right = right_line_finder.line_found()
        #print('\r' + '{:.2f}'.format(distance)+'flag:{}'.format(flag), end='', flush=True)
        if 0< distance <distance_T :
            flag = flag+1
        else:
            flag = flag

        if Final and left and right:
            car.steering(0,A_speed)
            time.sleep(0.5)
            car.stop()
            print('Success')
            break
        else:
            pass
        
        if not left and not right and flag<threshold:
            car.steering(0,A_speed+25)
        elif not left and right and flag<threshold:
            car.steering(0-A_diff,A_speed)  
            #car.trun_left(A_diff)
        elif left and not right and flag<threshold:
            car.steering(A_diff,A_speed)
            #car.trun_right(A_diff)
        elif flag>threshold:
            if turn_number<len(turnlist) and turnlist[turn_number] == 'right':
                car.right(0.3,87)
            elif turn_number<len(turnlist) and turnlist[turn_number] == 'left':
                car.left(0.3,91)
            else :
                print('----+----+----- Preset turn completed ----+---- Please Check ---+---+-')
                car.stop()
            turn_number = turn_number + 1
            flag = 0
            if turn_number == len(turnlist):#直角转弯结束寻找终点线
                Final = True
        else:
            pass
            #car.stop()

In [ ]:
#Top Level 函数
def Navigation(B_diff,B_speed,threshold,A_diff,A_speed,distance_T,turnlist):
    car.speed_offset = -1
    car.speed_diff = 0
    car.set_speed(80)  
    time.sleep(0.3)
    Navigation_B(B_diff,B_speed,threshold)
    Navigation_A(A_diff,A_speed,distance_T,threshold,turnlist)

In [ ]:
#中间巡线测试
btn3 = widgets.Button(description='PATH_B')
display(btn3)
def btn3_eventhandler(obj):
    time_start = time.time()
    Navigation_B(80,85,5)
    time_end = time.time()
    print('Time:{}'.format(time_end-time_start))
    
btn3.on_click(btn3_eventhandler)

In [ ]:
#两边巡线测试
turnlist=['right','left']
btn4 = widgets.Button(description='PATH_A')
display(btn4)
def btn4_eventhandler(obj):
    time_start = time.time()
    Navigation_A(28,70,16,3,turnlist)
    time_end = time.time()
    print('Time:{}'.format(time_end-time_start))
    
btn3.on_click(btn3_eventhandler)

In [ ]:
#综合测试
turnlist=['right','left']
btn5 = widgets.Button(description='GO')
display(btn5)
def btn5_eventhandler(obj):
    time_start = time.time()
    Navigation(B_diff=70,B_speed=70,threshold=5,
               A_diff=28,A_speed=70,distance_T=18,turnlist)
    time_end = time.time()
    print('Time:{}'.format(time_end-time_start))
    
btn5.on_click(btn5_eventhandler)

In [ ]:
#停止Car
btn0 = widgets.Button(description='STOP！！！')
display(btn0)
def btn0_eventhandler(obj):
    car.stop()
    
btn0.on_click(btn0_eventhandler)